In [1]:
from methods import load_data, select_route_data, group_nodes, group_links, obtain_routes, export_to_json, qgis_json_format, to_list, create_volume_table
import pathlib

In [2]:
strategic_data_file = pathlib.Path.cwd() / "Enfield Saturn Routes AM test.xlsx"
qgis_file = pathlib.Path.cwd() / "qgis_data.xlsx"
strategic_raw_data, qgis_table = load_data(strategic_data_file, qgis_file) #Load raw data

In [3]:
volumes, nodes = select_route_data(strategic_raw_data)
ogv_volumes, ogv_nodes = select_route_data(strategic_raw_data, ogv=True) #Extract routes for user class 9 (OGVs)

In [4]:
data_sets = {"routes": nodes, "ogv_routes": ogv_nodes}
route_codes, all_routes_list=[],[]
for user_class, data in data_sets.items():
    nodes = to_list(data)
    nodes_grouped=group_nodes(nodes) # Group the node sequences that make up a route
    links = group_links(nodes_grouped) # From the node sequences create the links
    routes = obtain_routes(links, qgis_table) # For the links create the list of links that make up each route
    all_routes_list.append(routes) # List of all routes, before dropping duplicates, used for volume results
    qgis_routes, route_ids = qgis_json_format(routes) if user_class != "ogv_routes" else qgis_json_format(routes, ogv=True) # Format results
    route_codes.append([route_ids[i].split("/")[-1].strip(".gpkg") for i in range(len(routes))])
    export_to_json(user_class, qgis_routes) # Export json files

In [5]:
create_volume_table(route_codes, all_routes_list, all_volumes = [volumes, ogv_volumes]) # Create a table of routes and volumes and export it